In [1]:
import pandas as pd
import numpy as np

In [2]:
from env import googleapi

In [3]:
from langchain import PromptTemplate, LLMChain

In [4]:
from markdownify import markdownify as md

def unmarkdown(text):
    return md(text)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

### feeding contracts to llms for RFP generation

### prompts

In [25]:
deepseek_template = """
You are an expert contract analyst. Analyze the provided contract document and extract ALL information relevant to creating an 
RFP (Request for Proposal). 

Here is the contract information: {contract}
Focus on extracting:
1. Project scope and deliverables
2. Technical requirements and specifications
3. Timeline constraints and key deadlines
4. Budget parameters and payment terms
5. Compliance requirements and regulatory standards
6. Evaluation criteria from previous contracts
7. Stakeholder information and points of contact
8. Historical performance metrics and benchmarks
9. Risk factors and mitigation strategies

Format your response as structured data with clear sections for each category above. Include ONLY factual information directly stated or 
clearly implied in the contract. If information for any category is missing, explicitly note its absence.
For each extracted piece of information, include the specific section or page reference from the source document.

-- If you find very less information you can add information by your side but assuming lesser information as base.
"""

deepseek_prompt = PromptTemplate.from_template(deepseek_template)

In [26]:
gemini_template = """You are an RFP structure specialist. Using the extracted contract data provided below, 
create a comprehensive outline/structure for a Request for Proposal document.

{out_deepseek}

Your task is to:
1. Organize the extracted information into a logical RFP framework
2. Create section headings and subheadings that follow standard RFP conventions
3. Provide brief descriptions for what each section should contain
4. Identify any critical gaps in the provided information that need to be addressed
5. Suggest appropriate placement for evaluation criteria and scoring mechanisms

Format your response as a detailed outline with hierarchical numbering and clear instructions for what information belongs in each section. 
Focus only on creating the structure without filling in detailed content.
"""

gemini_prompt = PromptTemplate.from_template(gemini_template)

In [42]:
gemini_template2 = """You are an expert contract analyst specialized in RFP development. Analyze the provided contract thoroughly and extract all information needed to create a comprehensive Request for Proposal.
Here is the contract information : 
{contract}

Your analysis should identify and extract:

1. PROJECT FUNDAMENTALS:
   - Core project objectives and scope
   - Specific deliverables with detailed specifications
   - Technical requirements with measurable parameters

2. OPERATIONAL PARAMETERS:
   - Timeline requirements including milestones and deadlines
   - Budget constraints and payment structures
   - Resource requirements and allocation guidelines

3. COMPLIANCE FRAMEWORK:
   - Regulatory requirements and standards
   - Required certifications and qualifications
   - Legal constraints and obligations

4. EVALUATION FRAMEWORK:
   - Prior vendor selection criteria
   - Performance metrics and KPIs
   - Critical success factors

For each extracted element, include:
- Direct quotation from the contract (when available)
- Location reference (section/page number)
- Level of importance (Critical, High, Medium, Low)

Organize your output in a structured format that can be directly used as input for RFP creation. If certain information is missing from the contract but typically needed in an RFP, explicitly note these gaps.

-- If you find very less information you can add information by your side but assuming lesser information as base.
-- also dont give reponse like you have made it, just what is instructed dont leave blank spaces, make up things if doesnt exist
"""

gemini_prompt2 = PromptTemplate.from_template(gemini_template2)

In [43]:
llama_template = """You are an RFP content development expert. Below is an outline structure for an RFP and the original contract data extraction. 
Your task is to expand this outline into a comprehensive draft RFP by adding appropriate detail and professional language.

RFP STRUCTURE:
{out_gemini}

ORIGINAL CONTRACT DATA:
{contract}

For each section in the outline:
1. Draft formal RFP language that incorporates the relevant extracted information
2. Add standard RFP phrasing and terminology where appropriate
3. Expand on technical requirements with industry-standard specificity
4. Include clear evaluation criteria and response guidelines for vendors
5. Add appropriate disclaimers and legal language
6. Ensure consistent formatting and professional tone throughout

Your output should be a complete, professional RFP draft that follows the provided structure while incorporating all relevant contract data in a 
cohesive document.

"""

llama_prompt = PromptTemplate.from_template(llama_template)

In [79]:
llama_template2 = """
You are an expert RFP development specialist. Transform the provided contract analysis into a complete, professional Request for Proposal document.

CONTRACT ANALYSIS:
{gemini_out}

Create a comprehensive RFP with all standard sections including executive summary, detailed requirements, vendor response guidelines, evaluation criteria, and submission instructions.

The RFP should:
1. Be completely ready for distribution to vendors without further editing
2. Include all information from the contract analysis in appropriate sections
3. Follow standard RFP formatting and structure
4. Use precise, professional language throughout
5. Fill any information gaps with standard industry practices

Your output should be a polished, professional procurement document that follows best practices for RFP development in this industry.
-- also dont give reponse like you have made it, just what is instructed and dont leave blank spaces, make up things if doesnt exist

"""
llama_prompt2 = PromptTemplate.from_template(llama_template2)

In [45]:
gemma_template = """You are an RFP quality assurance specialist. Review the draft RFP provided below and refine it to ensure it meets the highest professional standards.

{out_llama}

Your review and refinement should:
1. Identify and correct any inconsistencies, ambiguities, or gaps in the content
2. Enhance clarity and precision of language, especially in technical specifications
3. Strengthen evaluation criteria to ensure they are measurable and objective
4. Verify that all timeline and deadline information is logically consistent
5. Ensure compliance requirements are comprehensive and clearly stated
6. Add any missing standard RFP components (e.g., submission instructions, Q&A process)
7. Improve overall document structure, flow, and readability
8. Format the document according to professional RFP standards

Your output should be a final, polished RFP that is ready for distribution to potential vendors.
Focus on making substantive improvements rather than minor stylistic changes."""

gemma_prompt = PromptTemplate.from_template(gemma_template)

In [83]:
gemma_template2 = """
You are an elite RFP development specialist. Using the contract analysis provided below, create a complete, professional-grade Request for Proposal (RFP) document ready for distribution to vendors.

CONTRACT ANALYSIS:
{llama_out}

Create a comprehensive RFP that includes:

1. EXECUTIVE SUMMARY:
   - Concise overview of the project purpose and objectives
   - Summary of key requirements and timeline

2. DETAILED RFP BODY:
   - Background and context
   - Comprehensive scope of work
   - Detailed technical specifications
   - Complete timeline with specific milestones
   - Deliverables with acceptance criteria
   - Budget parameters and payment terms
   - Compliance requirements and regulatory standards

3. VENDOR RESPONSE REQUIREMENTS:
   - Required proposal structure
   - Technical response guidelines
   - Pricing table format
   - Qualification and certification requirements
   - References and past performance evidence

4. EVALUATION FRAMEWORK:
   - Detailed evaluation criteria with weightings
   - Scoring methodology
   - Selection process description

5. SUBMISSION GUIDELINES:
   - Submission deadlines and format
   - Q&A process
   - Pre-proposal conference details (if applicable)
   - Contact information

Format the document professionally with:
- Numbered sections and subsections
- Proper headings and subheadings
- Tables for technical specifications
- Bullet points for lists
- Highlighted key dates and requirements

IMPORTANT INSTRUCTIONS:
- Do NOT leave placeholders like "Company Name" or "Insert Timeline Here".
- If any information is missing, invent realistic and context-appropriate content to complete the section.
- Use formal, precise language suitable for official procurement.
- Ensure the final RFP is complete, polished, and ready for distribution without any further edits.

Only return the final RFP text. Do not include any meta-comments or apologies.
"""
gemma_prompt2 = PromptTemplate.from_template(gemma_template2)

In [84]:
import os
os.environ["GOOGLE_API_KEY"] = googleapi

In [48]:
def gemini_call(deepseek_out, gemini_prompt):
    # Instantiate the Gemini model (e.g., gemini-pro or gemini-2.0-flash-001)
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17", temperature=0.7)
    # Create an LLMChain with the LLM and prompt
    chain = LLMChain(llm=llm, prompt=gemini_prompt)
    # Run the chain with your input
    response = chain.run(contract = deepseek_out)
    response = unmarkdown(response)
    #print(response)
    return response

In [70]:
def gemini_call2(deepseek_out, gemini_prompt):
    # Instantiate the Gemini model (e.g., gemini-pro or gemini-2.0-flash-001)
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17", temperature=0.7)
    # Create an LLMChain with the LLM and prompt
    chain = LLMChain(llm=llm, prompt=gemini_prompt)
    # Run the chain with your input
    response = chain.run(llama_out = deepseek_out)
    response = unmarkdown(response)
    #print(response)
    return response

In [30]:
def ollama_call(arg, out, prompt, model, extra = None):
    if not extra:
        response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt.format(arg = out)}])
    else:
        response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt.format(arg = out)}])
        
    #responses.append([model, response])
    if model in ["qwen3:4b",  "deepseek-r1:7b"]:
        response = response["message"]["content"].split("</think>")[1]
    else:
        response = response["message"]["content"]   
    response = unmarkdown(response)
    return response

In [19]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [21]:
train_df.shape, test_df.shape

((417, 2), (93, 2))

In [22]:
import ollama
models = ["deepseek-r1:7b","gemini2.5",  "llama3.2", "gemma3:4b"] 

In [39]:
def make_rfp(df):
    deepseek_outs, gemini_outs, llama_outs = [], [], []
    gemma_outs = []
    o = 0
    while o < len(df):
        print("on", o)
        contract = df["contract"][o]
        
        deepseek_out = ollama.chat(model = "deepseek-r1:7b", messages = [{"role": "user", "content": deepseek_template.format(contract = contract)}])
        deepseek_out = deepseek_out["message"]["content"].split("</think>")[1]
        deepseek_outs.append(deepseek_out)
        
        gemini_out = gemini_call(deepseek_out, gemini_prompt)
        #gemini_out = gemini_out["message"]["content"]
        gemini_outs.append(gemini_out)
        
        llama_out = ollama.chat(model = "llama3.2", messages = [{"role": "user", "content": llama_template.format(out_gemini = gemini_out, contract = contract)}])
        llama_out = llama_out["message"]["content"]
        llama_outs.append(llama_out)
        
        gemma_out = ollama.chat(model = "gemma3:4b", messages = [{"role": "user", "content": gemma_template.format(out_llama = llama_out)}])
        gemma_out = gemma_out["message"]["content"]
        gemma_outs.append(gemma_out)
        o +=1
        
    return deepseek_outs, gemini_outs, llama_outs, gemma_outs

In [105]:
def save(l, o):
    df = pd.DataFrame()
    df["analysis"] = l[0]
    df["skeleton"] = l[1]
    df["final"] = l[2]
    df.to_csv(r"C:\Users\91962\voluntas2\rfps_csv\test\\" + f"upto_{o}.csv", index = False)

In [106]:
save([["1"], ["2"], ["3"]], 9)

In [102]:
def make_rfp2(df):
    gemini2_outs, gemini_outs, llama_outs = [], [], []
    o = 0
    while o < len(df):
        #print("on", o)
        contract = df["contract"][o]
        """
        deepseek_out = ollama.chat(model = "deepseek-r1:7b", messages = [{"role": "user", "content": deepseek_template.format(contract = contract)}])
        deepseek_out = deepseek_out["message"]["content"].split("</think>")[1]
        deepseek_outs.append(deepseek_out)
        """
        print("on", o, "/", len(df))
        print("gemini")
        
        gemini_out = gemini_call(contract, gemini_prompt2)
        #gemini_out = gemini_out["message"]["content"]
        gemini_outs.append(gemini_out)

        print("llama")
        llama_out = ollama.chat(model = "llama3.2", messages = [{"role": "user", "content": llama_template2.format(gemini_out = gemini_out)}])
        llama_out = llama_out["message"]["content"]
        llama_outs.append(llama_out)
        
        print("gemini2")
        gemini_out2 = gemini_call2(llama_out, gemma_prompt2)
        #gemini_out = gemini_out["message"]["content"]
        gemini2_outs.append(gemini_out2)
        """
        gemma_out = ollama.chat(model = "phi3:mini", messages = [{"role": "user", "content": gemma_template2.format(llama_out = llama_out)}])
        gemma_out = gemma_out["message"]["content"]
        gemma_outs.append(gemma_out)
        o +=1
        """
        if o < 3:
            print(gemini_out)
            print("_"*100)
            print(llama_out)
            print("_"*100)
            print(gemini_out2)
            print("_"*100)

        if o % 20 == 0:
            save([gemini_outs, llama_outs, gemini2_outs], o)
            print("saved upto", o, "index")
        o +=1
    return gemini_outs, llama_outs, gemini2_outs

In [103]:
# for training data
trainx = make_rfp2(train_df)

on 0 / 417
gemini
llama
gemini2
Okay, here is a thorough analysis of the provided Chase Affiliate Agreement, extracting the information needed to create a comprehensive Request for Proposal (RFP) for potential Affiliates.
This analysis assumes the RFP is being issued by Chase to solicit potential Affiliates who would then enter into an agreement \*like\* the one provided.
---
\*\*RFP INPUT DATA EXTRACTION: CHASE AFFILIATE PROGRAM\*\*
\*\*Source Document:\*\* Exhibit 10.33, Chase Affiliate Agreement, Last Updated: April 6, 2007
\*\*1. PROJECT FUNDAMENTALS\*\*
\* \*\*Core Project Objectives and Scope:\*\*
\* \*\*Description:\*\* Establish and maintain links from an Affiliate's website to Chase.com to drive applications for Chase credit cards. Chase processes applications and pays commissions for approved accounts.
\* \*\*Direct Quotation:\*\* "Once accepted into the Affiliate Program, an Affiliate can establish links from the Affiliate’s Website to [Chase.com]. Chase will pay Affiliate a

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


llama
gemini2
saved upto 340 index
on 341 / 417
gemini
llama
gemini2
on 342 / 417
gemini
llama
gemini2
on 343 / 417
gemini
llama
gemini2
on 344 / 417
gemini
llama
gemini2
on 345 / 417
gemini
llama
gemini2
on 346 / 417
gemini
llama
gemini2
on 347 / 417
gemini
llama
gemini2
on 348 / 417
gemini
llama
gemini2
on 349 / 417
gemini
llama
gemini2
on 350 / 417
gemini
llama
gemini2
on 351 / 417
gemini
llama
gemini2
on 352 / 417
gemini
llama
gemini2
on 353 / 417
gemini
llama
gemini2
on 354 / 417
gemini
llama
gemini2
on 355 / 417
gemini
llama
gemini2
on 356 / 417
gemini
llama
gemini2
on 357 / 417
gemini
llama
gemini2
on 358 / 417
gemini
llama
gemini2
on 359 / 417
gemini
llama
gemini2
on 360 / 417
gemini
llama
gemini2
saved upto 360 index
on 361 / 417
gemini
llama
gemini2
on 362 / 417
gemini
llama
gemini2
on 363 / 417
gemini
llama
gemini2
on 364 / 417
gemini
llama
gemini2
on 365 / 417
gemini
llama
gemini2
on 366 / 417
gemini
llama
gemini2
on 367 / 417
gemini
llama
gemini2
on 368 / 417
gemini
llama


In [104]:
save(trainx, 417)

## started on 18.50  finished on 10.30 second day ⬆️⬆️⬆️
## started on 10.40⬇️⬇️⬇️

In [108]:
# for training data
testx = make_rfp2(test_df)

on 0 / 93
gemini
llama
gemini2
Okay, here is a comprehensive analysis of the provided Marketing Affiliate Agreement, extracted and structured as input for creating a Request for Proposal (RFP). The RFP would be issued by Birch First Global Investments Inc. (or a related entity) seeking a new Marketing Affiliate to perform the services outlined in this agreement.
\*\*RFP INPUT DATA FROM MARKETING AFFILIATE AGREEMENT (Exhibit 10.27)\*\*
\*\*Contract Source:\*\* CYBERGY HOLDINGS, INC., 10-Q, 5/20/2014
\*\*Original Agreement Parties:\*\*
\* Company: Birch First Global Investments Inc. (US Virgin Islands)
\* Marketing Affiliate (MA): Mount Knowledge Holdings Inc. (Nevada)
\*\*Original Agreement Date:\*\* May 8, 2014
\*\*Subject of RFP:\*\* Seeking a Marketing Affiliate to provide marketing, sales, and support services for Company's Technology (specifically ECO related platform and content).
---
\*\*1. PROJECT FUNDAMENTALS\*\*
\* \*\*Core Project Objectives and Scope:\*\*
\* \*\*Objective:\*

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


on 10 / 93
gemini
llama
gemini2
on 11 / 93
gemini
llama
gemini2
on 12 / 93
gemini
llama
gemini2
on 13 / 93
gemini
llama
gemini2
on 14 / 93
gemini
llama
gemini2
on 15 / 93
gemini
llama
gemini2
on 16 / 93
gemini
llama
gemini2
on 17 / 93
gemini
llama
gemini2
on 18 / 93
gemini
llama
gemini2
on 19 / 93
gemini
llama
gemini2
on 20 / 93
gemini
llama
gemini2
saved upto 20 index
on 21 / 93
gemini
llama
gemini2
on 22 / 93
gemini
llama
gemini2
on 23 / 93
gemini
llama
gemini2
on 24 / 93
gemini
llama
gemini2
on 25 / 93
gemini
llama
gemini2
on 26 / 93
gemini
llama
gemini2
on 27 / 93
gemini
llama
gemini2
on 28 / 93
gemini
llama
gemini2
on 29 / 93
gemini
llama
gemini2
on 30 / 93
gemini
llama
gemini2
on 31 / 93
gemini
llama
gemini2
on 32 / 93
gemini
llama
gemini2
on 33 / 93
gemini
llama
gemini2
on 34 / 93
gemini
llama
gemini2
on 35 / 93
gemini
llama
gemini2
on 36 / 93
gemini
llama
gemini2
on 37 / 93
gemini
llama
gemini2
on 38 / 93
gemini
llama
gemini2
on 39 / 93
gemini
llama
gemini2
on 40 / 93
gemini
ll

In [109]:
save(testx, 93)